In [1]:
import pandas as pd
import numpy as np

from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, List, Optional, Tuple

import muspy

In [2]:
PROJECT_ROOT = Path.cwd()
DATA_ROOT = PROJECT_ROOT / "data" / "essen"
DATA_ROOT.mkdir(parents=True, exist_ok=True)

ds = muspy.datasets.EssenFolkSongDatabase(
    root=Path("data/essen"),
    download_and_extract=True,
    convert=False,
    use_converted=None,
    verbose=True,
    n_jobs=-1
)

Skip downloading as the `.muspy.success` file is found.
Skip extracting as the `.muspy.success` file is found.


In [22]:
NAME_MUSIC = 'essen'
DATA_TOKENS = f"data/songs_token_{NAME_MUSIC}.json"
CHORDS = 'chords_experiments'

Переводим песни в формат токенов и сохраняем

In [7]:
import json
def music_to_tokens(music: muspy.Music, step_division: int = 8):

    step_ticks = max(1, int(music.resolution) // step_division)
    tracks = [t for t in music.tracks if t.notes]
    track = max(tracks, key=lambda t: len(t.notes))

    notes = sorted(track.notes, key=lambda n: n.time)

    tokens = []
    prev_end = 0

    for n in notes:
        rest_ticks = max(0, int(n.time) - int(prev_end))

        rest = int(np.round(rest_ticks / step_ticks)) # это пауза перед нотой в шагах
        dur = max(1, int(np.round(int(n.duration) / step_ticks))) # это длительность ноты в шагах
        pitch = int(n.pitch) # это значение MIDI ноты (от 0 до 127)

        tokens.append((pitch, dur, rest))
        prev_end = max(prev_end, int(n.time) + int(n.duration))

    return tokens

songs = []
for i in range (1000, 2000, 1):
    try:
        music = ds[i]
    except:
        continue
    music = ds[i]
    tokens = music_to_tokens(music)
    songs.append(tokens)

print("Количество песен:", len(songs))



def save_songs_simple(songs, filename=DATA_TOKENS):
    """Самый простой вариант - сохраняем как есть"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(songs, f)

    print(f"✓ Сохранено {len(songs)} песен в {filename}")
    return filename

def load_songs_simple(filename):
    """Загружает песни из JSON файла"""
    with open(filename, 'r', encoding='utf-8') as f:
        songs = json.load(f)

    print(f"✓ Загружено {len(songs)} песен из {filename}")
    return songs


# можно сохранить чтобы не перекачивать датасет
# save_songs_simple(songs)
# songs = load_songs_simple(DATA_TOKENS)

Количество песен: 1000


Генерируем марковскими цепями мелодию

In [9]:
from collections import defaultdict, Counter


def make_markov_chain(songs):
    transitions = defaultdict(Counter)  # матрица переходов
    starts = Counter()

    for tokens in songs:
        starts[tokens[0]] += 1
        for a, b in zip(tokens, tokens[1:]):
            transitions[a][b] += 1

    return starts, transitions


import random


def sample_from_counter(counter: Counter):
    items = list(counter.items())

    states, weights = [], []
    for s, w in items:
        states.append(s)
        weights.append(w)
    sum_weights = sum(weights)

    return random.choices(states, weights=np.array(weights) / sum_weights)[0]


def generate_markov(starts, trans, length=200):
    cur = sample_from_counter(starts)
    out = [cur]
    for _ in range(length - 1):
        if cur not in trans or len(trans[cur]) == 0:
            cur = sample_from_counter(starts)
        else:
            cur = sample_from_counter(trans[cur])
        out.append(cur)
    return out


starts, trans = make_markov_chain(songs)
gen_tokens = generate_markov(starts, trans, length=200)

print("Сгенерировано токенов:", len(gen_tokens))
print("Первые 10:", gen_tokens[:10])



Сгенерировано токенов: 200
Первые 10: [(62, 4, 0), (67, 4, 0), (67, 12, 0), (71, 4, 0), (72, 2, 0), (69, 2, 0), (67, 2, 0), (71, 2, 0), (69, 4, 0), (67, 8, 0)]


In [10]:
def tokens_to_midi(tokens, out_midi="markov.mid", resolution=480, step_division=8):
    step_ticks = resolution // step_division
    music = muspy.Music(resolution=resolution)
    track = muspy.Track(program=102)
    music.tracks.append(track)

    t = 0
    for pitch, dur, rest in tokens:
        t += int(rest) * step_ticks
        track.notes.append(muspy.Note(time=t, pitch=int(pitch), duration=int(dur) * step_ticks, velocity=80))
        t += int(dur) * step_ticks

    muspy.write_midi(out_midi, music)
    return out_midi


tokens_to_midi(gen_tokens, f"markov_{NAME_MUSIC}.mid")



'markov_essen.mid'

Учимся сохранять аккомпонимент

In [12]:
def save_with_accompaniment(melody_tokens, accompaniment_tokens=None, filename=f"markov_akk_{NAME_MUSIC}.mid"):
    """
    Сохраняет мелодию и аккомпанемент в один трек
    """
    music = muspy.Music(resolution=480)
    track = muspy.Track(program=0)  # фортепиано
    music.tracks.append(track)

    # Добавляем мелодию
    t = 0
    for pitch, dur, rest in melody_tokens:
        t += int(rest) * 120
        track.notes.append(muspy.Note(
            time=t,
            pitch=int(pitch),
            duration=int(dur) * 120,
            velocity=80  # громкая - мелодия
        ))
        t += int(dur) * 120

    # Добавляем аккомпанемент (если есть)
    if accompaniment_tokens:
        t = 0
        for acc_note in accompaniment_tokens:
            # acc_note может быть: (pitch, duration, rest) или (type, pitch, duration, offset)
            if len(acc_note) == 3:
                pitch, dur, rest = acc_note
                offset = 0
            elif len(acc_note) == 4:
                _, pitch, dur, offset = acc_note

            t += int(rest) * 120 if len(acc_note) >= 3 else 0
            track.notes.append(muspy.Note(
                time=t + int(offset) * 120,
                pitch=int(pitch),
                duration=int(dur) * 120,
                velocity=50  # тише - аккомпанемент
            ))
            t += int(dur) * 120

    muspy.write_midi(filename, music)
    print(f"✓ MIDI с аккомпанементом сохранен: {filename}")
    return filename

### Аккомпанимент

#### 1. Аккорды алгосом

In [13]:
import random
import numpy as np
from collections import defaultdict, Counter

def detect_key(tokens):
    """Определяет тональность по нотам"""
    pitches = [token[0] for token in tokens if isinstance(token, tuple)]
    pitch_counts = Counter(pitches)

    # Простая эвристика - самая частая нота как тоника
    if pitch_counts:
        most_common = pitch_counts.most_common(1)[0][0]
        # Округляем до ближайшей C (до) или A (ля) для мажора/минора
        tonic = most_common % 12
        # Простая проверка мажор/минор
        major_intervals = [0, 2, 4, 5, 7, 9, 11]  # C major scale
        if tonic in major_intervals:
            return ('C', 'major') if tonic == 0 else ('unknown', 'major')
        return ('A', 'minor') if tonic == 9 else ('unknown', 'minor')
    return ('C', 'major')  # По умолчанию

def get_chord_progression(tokens, key='C', mode='major'):
    """Генерирует простую аккордовую последовательность"""
    # Основные аккорды тональности
    if mode == 'major':
        # I, ii, iii, IV, V, vi, vii°
        chords = {
            0: [60, 64, 67],  # C
            2: [62, 65, 69],  # Dm
            4: [64, 67, 71],  # Em
            5: [65, 69, 72],  # F
            7: [67, 71, 74],  # G
            9: [69, 72, 76],  # Am
            11: [71, 74, 77]  # Bdim
        }
    else:  # minor
        chords = {
            9: [57, 60, 64],  # Am
            11: [59, 62, 65], # Bdim
            0: [60, 63, 67],  # C
            2: [62, 65, 68],  # Dm
            4: [64, 67, 70],  # Em
            5: [65, 68, 72],  # F
            7: [67, 70, 74]   # G
        }

    progression = []
    for i, token in enumerate(tokens):
        if isinstance(token, tuple):
            pitch = token[0]
            scale_degree = (pitch - 60) % 12  # C=60

            # Меняем аккорд каждые 4 ноты или на сильных долях
            if i % 4 == 0 or (len(progression) == 0):
                # Выбираем аккорд, содержащий текущую ноту
                possible_chords = []
                for degree, chord_notes in chords.items():
                    if (scale_degree - degree) % 12 in [0, 4, 7, 3]:  # Ноты аккорда
                        possible_chords.append(chord_notes)

                if possible_chords:
                    progression.append(random.choice(possible_chords))
                else:
                    # Аккорд I ступени по умолчанию
                    progression.append(chords[0])

    return progression

In [ ]:
accomaniment = get_chord_progression(gen_tokens)
save_with_accompaniment(melody_tokens=gen_tokens, accompaniment_tokens=accomaniment, filename=f"{CHORDS}/markov_akk1_{NAME_MUSIC}.mid")


2. Аккорд каждые n нот - мажорные трезвучия

In [15]:
# НОВАЯ ФУНКЦИЯ: Простая гармония (аккорды раз в N нот)
def add_chords_to_melody(melody_tokens, chord_interval=4):
    """
    Добавляет аккорды к мелодии.
    Возвращает список всех нот (мелодия + аккорды) для одного трека.
    """
    all_notes = []

    for i, (pitch, dur, rest) in enumerate(melody_tokens):
        # Добавляем ноту мелодии
        all_notes.append(('melody', pitch, dur, rest))

        # Добавляем аккорд каждые chord_interval нот
        if i % chord_interval == 0:
            # Строим простой мажорный аккорд от ноты мелодии
            base_pitch = int(pitch)

            # Три ноты аккорда (мажорное трезвучие)
            chord_notes = [
                base_pitch,        # основной тон
                base_pitch + 4,    # большая терция
                base_pitch + 7     # чистая квинта
            ]

            # Добавляем каждую ноту аккорда
            for chord_note in chord_notes:
                # Аккорд звучит дольше (в 2 раза дольше ноты мелодии)
                all_notes.append(('chord', chord_note, dur * 3, 0))

    return all_notes

# НОВАЯ ФУНКЦИЯ: Сохранение в один трек (и мелодия, и аккорды)
def save_all_in_one_track(all_notes, filename="complete_music.mid"):
    """
    Сохраняет все ноты (мелодию и аккорды) в один трек.
    Мелодия громче, аккорды тише - всё слышно!
    """
    music = muspy.Music(resolution=480)
    track = muspy.Track(program=0)  # фортепиано
    music.tracks.append(track)

    t = 0
    for note_type, pitch, dur, rest in all_notes:
        # Учитываем паузу
        t += int(rest) * 120

        # Настраиваем громкость: мелодия громче, аккорды тише
        velocity = 80 if note_type == 'melody' else 60

        # Добавляем ноту
        track.notes.append(muspy.Note(
            time=t,
            pitch=int(pitch),
            duration=int(dur) * 120,
            velocity=velocity
        ))

        # Для мелодии двигаем время вперед
        # Для аккордов - нет, они звучат параллельно
        if note_type == 'melody':
            t += int(dur) * 120

    muspy.write_midi(filename, music)
    print(f"✓ Готово! Сохранено: {filename}")
    print(f"  Всего нот: {len(all_notes)}")
    print(f"  Фортепиано, один трек, всё слышно!")
    return filename

In [ ]:
all_notes = add_chords_to_melody(gen_tokens)
save_all_in_one_track(all_notes,filename=f'{CHORDS}/markov_akk2_{NAME_MUSIC}.mid')

3. Очень умные аакорды (по мелодии определяют тональность + паттерны по стилю музыки)

In [37]:
import random

def detect_scale_type(pitches):
    """Определяет мажор или минор по нотам"""
    pitch_classes = set([p % 12 for p in pitches])

    # Проверяем наличие характерных интервалов
    major_third = any((p + 4) % 12 in pitch_classes for p in pitch_classes)
    minor_third = any((p + 3) % 12 in pitch_classes for p in pitch_classes)

    if major_third and not minor_third:
        print('major')
        return 'major'
    elif minor_third and not major_third:
        print('minor')
        return 'minor'
    else:
        return random.choice(['major', 'minor'])

def get_chord_progression_advanced(melody_tokens, style='folk'):
    """
    Продвинутая генерация аккордовой последовательности
    """
    pitches = [p for p, _, _ in melody_tokens]
    scale_type = detect_scale_type(pitches)

    # Определяем тонику (самая частая нота)
    tonic = max(set([p % 12 for p in pitches]),
                key=lambda x: [p % 12 for p in pitches].count(x))

    if scale_type == 'major':
        # I - ii - iii - IV - V - vi - vii°
        chords_by_degree = {
            0: [tonic, tonic + 4, tonic + 7],      # I
            2: [tonic + 2, tonic + 5, tonic + 9],  # ii
            4: [tonic + 4, tonic + 7, tonic + 11], # iii
            5: [tonic + 5, tonic + 9, tonic + 0],  # IV
            7: [tonic + 7, tonic + 11, tonic + 2], # V
            9: [tonic + 9, tonic + 0, tonic + 4],  # vi
            11: [tonic + 11, tonic + 2, tonic + 5] # vii°
        }

        # Популярные последовательности для народной музыки
        if style == 'folk':
            progressions = [
                [0, 5, 7, 0],      # I-IV-V-I (самая популярная)
                [0, 4, 5, 0],      # I-iii-IV-I
                [0, 5, 9, 7],      # I-IV-vi-V
                [0, 5, 0, 7],      # I-IV-I-V
            ]

        elif style == 'pop':
            progressions = [
                [0, 5, 9, 4],      # I-IV-vi-iii
                [0, 5, 9, 7],      # I-IV-vi-V
                [0, 5, 7, 9],      # I-IV-V-vi
            ]

    else:  # minor
        chords_by_degree = {
            0: [tonic, tonic + 3, tonic + 7],      # i
            2: [tonic + 2, tonic + 5, tonic + 8],  # ii°
            3: [tonic + 3, tonic + 7, tonic + 10], # III
            5: [tonic + 5, tonic + 8, tonic + 0],  # iv
            7: [tonic + 7, tonic + 10, tonic + 2], # v
            8: [tonic + 8, tonic + 0, tonic + 3],  # VI
            10: [tonic + 10, tonic + 2, tonic + 5] # VII
        }

        progressions = [
            [0, 5, 7, 0],      # i-iv-v-i
            [0, 3, 5, 7],      # i-III-iv-v
            [0, 3, 7, 5],      # i-III-v-iv
        ]

    # Выбираем случайную последовательность
    progression = random.choice(progressions)

    # Преобразуем в аккорды
    chords = []
    for degree in progression:
        if degree in chords_by_degree:
            # Транспонируем аккорд
            chord = [note + 60 for note in chords_by_degree[degree]]
            chords.append(chord)

    return chords



# def create_all_notes_with_harmony(melody_tokens, style='folk', chord_interval=4):
#     """
#     Создает список all_notes с мелодией и гармонизированными аккордами
#
#     Возвращает:
#         all_notes = список кортежей (type, pitch, duration, rest, velocity)
#         chords = список аккордов (для информации)
#     """
#     # 1. Получаем аккордовую последовательность
#     chords = get_chord_progression_advanced(melody_tokens, style)
#
#     # 2. Создаем список всех нот
#     all_notes = []
#
#     # 3. Добавляем мелодию (сначала просто копируем)
#     for pitch, dur, rest in melody_tokens:
#         all_notes.append(('melody', pitch, dur, rest))  # velocity=80
#
#     # 4. Добавляем аккорды на сильные доли
#     for i in range(0, len(melody_tokens), chord_interval):
#         # Определяем какой аккорд играть
#         chord_idx = (i // chord_interval) % len(chords)
#         current_chord = chords[chord_idx]
#
#         # Вычисляем время начала аккорда (суммируем rest и dur предыдущих нот)
#         start_rest = 0
#         for j in range(i):
#             _, _, rest_j = melody_tokens[j]
#             start_rest += rest_j
#
#         # Для каждого тона аккорда добавляем отдельную запись
#         for chord_note in current_chord:
#             # Аккорды звучат дольше (умножаем длительность)
#             chord_duration = chord_interval * 2
#
#             # Добавляем аккордную ноту
#             all_notes.append(('chord', chord_note, chord_duration, start_rest))
#
#     return all_notes
def create_all_notes_with_sparse_chords(melody_tokens, style='folk', measure_length=8):
    """
    Аккорды только на первой доле каждого такта (редко)
    measure_length = сколько нот в одном такте
    """
    chords = get_chord_progression_advanced(melody_tokens, style)
    all_notes = []

    # 1. Мелодия
    for pitch, dur, rest in melody_tokens:
        all_notes.append(('melody', pitch, dur, rest))

    # 2. Время начала каждой ноты
    start_times = []
    t = 0
    for pitch, dur, rest in melody_tokens:
        start_times.append(t)
        t += (int(rest) + int(dur)) * 120

    # 3. Аккорды только на первой доле каждого такта
    for i in range(0, len(melody_tokens), measure_length):
        if i < len(start_times):
            chord_idx = (i // measure_length) % len(chords)
            current_chord = chords[chord_idx]

            start_time = start_times[i] // 120
            chord_duration = measure_length * 6  # почти весь такт

            for chord_note in current_chord:
                all_notes.append(('chord', chord_note, chord_duration, start_time))

                if i < measure_length * 3:  # отладка первых тактов
                    print(f"Такт {i//measure_length}: аккорд на ноте {i}, "
                          f"начало={start_time}, длит={chord_duration}")

    return all_notes

def save_all_in_one_track_fixed(all_notes, filename="complete_music.mid"):
    music = muspy.Music(resolution=480)
    track = muspy.Track(program=0)  # фортепиано
    music.tracks.append(track)

    # Словарь для группировки нот по времени
    notes_by_time = {}

    # Обрабатываем мелодию (последовательное время)
    current_time = 0
    for note_type, pitch, dur, rest in all_notes:
        if note_type == 'melody':
            # Для мелодии: добавляем паузу, затем ноту
            current_time += int(rest) * 120

            # Сохраняем в словарь по времени
            if current_time not in notes_by_time:
                notes_by_time[current_time] = []

            notes_by_time[current_time].append({
                'pitch': int(pitch),
                'duration': int(dur) * 120,
                'velocity': 80  # мелодия громче
            })

            current_time += int(dur) * 120

        else:  # chord
            # Для аккордов: rest - это АБСОЛЮТНОЕ время начала
            start_time = int(rest) * 120

            if start_time not in notes_by_time:
                notes_by_time[start_time] = []

            notes_by_time[start_time].append({
                'pitch': int(pitch),
                'duration': int(dur) * 120 * 6,
                'velocity': 65  # аккорды тише
            })

    # Добавляем все ноты в трек в правильном порядке времени
    for time in sorted(notes_by_time.keys()):
        for note_info in notes_by_time[time]:
            track.notes.append(muspy.Note(
                time=time,
                pitch=note_info['pitch'],
                duration=note_info['duration'],
                velocity=note_info['velocity']
            ))

    muspy.write_midi(filename, music)

    # Статистика
    melody_notes = len([n for n in all_notes if n[0] == 'melody'])
    chord_notes = len([n for n in all_notes if n[0] == 'chord'])

    print(f"✓ Сохранено: {filename}")
    print(f"  Мелодия: {melody_notes} нот")
    print(f"  Аккорды: {chord_notes} нот")
    print(f"  Всего уникальных временных точек: {len(notes_by_time)}")

    return filename

In [38]:
all_notes = create_all_notes_with_sparse_chords(gen_tokens)
save_all_in_one_track_fixed(all_notes,filename=f'{CHORDS}/markov_akk3_{NAME_MUSIC}.mid')

Такт 0: аккорд на ноте 0, начало=0, длит=48
Такт 0: аккорд на ноте 0, начало=0, длит=48
Такт 0: аккорд на ноте 0, начало=0, длит=48
Такт 1: аккорд на ноте 8, начало=32, длит=48
Такт 1: аккорд на ноте 8, начало=32, длит=48
Такт 1: аккорд на ноте 8, начало=32, длит=48
Такт 2: аккорд на ноте 16, начало=76, длит=48
Такт 2: аккорд на ноте 16, начало=76, длит=48
Такт 2: аккорд на ноте 16, начало=76, длит=48
✓ Сохранено: chords_experiments/markov_akk3_essen.mid
  Мелодия: 200 нот
  Аккорды: 75 нот
  Всего уникальных временных точек: 201


'chords_experiments/markov_akk3_essen.mid'

### Итого

1 - аккорды не получились

2 - просто базовые(мажорные) трезвучия от ноты - вечная классика(я так песни играю) - мне нравится!!!!

3 - какая-то умная теория - но так как мелодия то не умная а рандомная - не получается выделить красивые паттерны для аккордов и получается фальшь.